# Отчет

## Подход 1: градиентный бустинг "в лоб"

**1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?**

```python
['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']
```
В первые 5 минут матча может не произойти события взятия первой крови: 
```python
['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',]
```
То же с ward_time.

**2. Как называется столбец, содержащий целевую переменную?**
```python
'radiant_win'
```

**3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? **

train time = **98s**

AUC-ROC = **0.694529**

**4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев? **

Имеет, есть рост качества (см. n_estimators = 40). 
Чтобы ускорить обучения при увеличении количества деревьев можно можно уменьшить глубину деревьев.

## Подход 2: логистическая регрессия

**1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?**

train time = **11s**

AUC-ROC = **0.716500** (над всеми исходными признаками) (Выше, чем у град. бустинга)

Логистическая регрессия требует различных предобработок и эвристик чтобы показать приемлемые результаты.
Логистическая регрессия по сравнению с градиентным бустингом работает быстрее.

**2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?**

AUC-ROC = **0.716527** (без категориальных)

Для категориальных признаков в лог. регрессии нужна обработка (bags) т.к. их нельзя сравнивать между собой. Если обработка не используется (1 пункт.) то метод работает чуть лучше без этих параметров.

**3. Сколько различных идентификаторов героев существует в данной игре?**

108 идентификаторов

**4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?**

AUC-ROC = **0.752013** (с правильной обработкой кат. признаков)

Кач-во сильно улучшилось, т.к. начальный выбор героев сильно влияет на исход игры, нужно лишь правильно обработать эти данные.

**5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?**

Лучший алгоритм -- *linear regression, c=0.1, bag of words(cat features)*

min_pred = 0.0084676402042699675

max_pred = 0.99657066046098575

In [222]:
import pandas as pd
import numpy as np
import sklearn

print('python  v={} \npandas  v={} \nsklearn v={}'.format(3, pd.__version__, sklearn.__version__))

python  v=3 
pandas  v=0.18.1 
sklearn v=0.18


# Data preparation

In [2]:
features = pd.read_csv('features.csv')

In [6]:
drop_features = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire']

In [9]:
test_features = features.drop(drop_features, axis=1)

In [10]:
test_features.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [12]:
counters = test_features.count()

In [104]:
counters[counters < 97230].index.tolist()

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

In [17]:
test_features = test_features.fillna(0)

In [20]:
y = features['radiant_win']

# GradientBoostingClassifier

In [135]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [138]:
X = test_features.values
X.shape

(97230, 103)

In [139]:
import time

In [141]:
kf = KFold(n_splits=5, shuffle=True)
res = []
for n_trees in [10, 20, 30, 40]:
    btime = time.time()
    for train_indices, test_indices in kf.split(X, y):
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        clf = GradientBoostingClassifier(n_estimators=n_trees)
        clf.fit(X_train, y_train)

        pred = clf.predict_proba(X_test)[:, 1]
        score = roc_auc_score(y_test, pred)
        res.append({'roc_auc': score, 'n_trees': n_trees})
    print('n trees = {}, cv in {:.2f}'.format(n_trees, time.time() - btime))

n trees = 10, cv in 26.75
n trees = 20, cv in 50.42
n trees = 30, cv in 73.94
n trees = 40, cv in 98.42


## Result

In [142]:
res_grad_boost = pd.DataFrame(res)
res_grad_boost.groupby('n_trees').mean().sort_values('roc_auc', ascending=False)

,roc_auc
n_trees,
40,0.694529
30,0.690042
20,0.682059
10,0.663486


# linear model

In [37]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

## Naive approach

In [143]:
X = test_features.values
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [144]:
c_range = [10**i for i in range(-5, 3)]
c_range

[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

In [145]:
kf = KFold(n_splits=5, shuffle=True)

res = []
for c in c_range:
    tbegin = time.time()
    for train_indices, test_indices in kf.split(X, y):
        X_train, X_test = X_norm[train_indices], X_norm[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        clf = LogisticRegression(C=c)
        clf.fit(X_train, y_train)

        pred = clf.predict_proba(X_test)[:, 1]
        score = roc_auc_score(y_test, pred)
        res.append({'roc_auc': score, 'C': c})
    print('c = {} cv in {:.2f}'.format(c, time.time() - tbegin))

c = 1e-05 cv in 2.46
c = 0.0001 cv in 4.22
c = 0.001 cv in 8.38
c = 0.01 cv in 10.82
c = 0.1 cv in 10.73
c = 1 cv in 11.67
c = 10 cv in 11.37
c = 100 cv in 11.03


### Result

In [146]:
lin_regr_res = pd.DataFrame(res)
lin_regr_res.groupby('C').mean().sort_values('roc_auc', ascending=False)

,roc_auc
C,
0.01000,0.716500
100.00000,0.716466
1.00000,0.716458
0.10000,0.716413
10.00000,0.716352
0.00100,0.716275
0.00010,0.711324
0.00001,0.695124


## Remove cat. features

In [147]:
cat_features = ['lobby_type']
cat_features.extend(['r{}_hero'.format(i) for i in range(1, 6)])
cat_features.extend(['d{}_hero'.format(i) for i in range(1, 6)])

In [224]:
set(test_features['lobby_type'])

{0, 1, 7}

In [148]:
heroes_features = []
heroes_features.extend(['r{}_hero'.format(i) for i in range(1, 6)])
heroes_features.extend(['d{}_hero'.format(i) for i in range(1, 6)])

In [149]:
base_features = test_features.drop(cat_features, axis=1)
base_features = base_features.fillna(0)

X = base_features.values
X_norm = scaler.fit_transform(X)

In [153]:
X_norm.shape

(97230, 92)

In [154]:
kf = KFold(n_splits=5, shuffle=True)

res = []
for c in c_range:
    for train_indices, test_indices in kf.split(X, y):
        X_train, X_test = X_norm[train_indices], X_norm[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        clf = LogisticRegression(C=c)
        clf.fit(X_train, y_train)

        pred = clf.predict_proba(X_test)[:, 1]
        score = roc_auc_score(y_test, pred)
        res.append({'roc_auc': score, 'C': c})
    print('c', c)

c 1e-05
c 0.0001
c 0.001
c 0.01
c 0.1
c 1
c 10
c 100


### Result

In [155]:
lin_regr_res_wo_cat = pd.DataFrame(res)
lin_regr_res_wo_cat.groupby('C').mean().sort_values('roc_auc', ascending=False)

,roc_auc
C,
0.01000,0.716527
0.00100,0.716525
10.00000,0.716494
100.00000,0.716487
1.00000,0.716394
0.10000,0.716360
0.00010,0.711362
0.00001,0.695153


## One hot cat. features

In [127]:
data = test_features[heroes_features]

In [172]:
len(set(test_features['r1_hero']))  # actual number of heroes

108

In [162]:
data.max()

r1_hero    112
r2_hero    112
r3_hero    112
r4_hero    112
r5_hero    112
d1_hero    112
d2_hero    112
d3_hero    112
d4_hero    112
d5_hero    112
dtype: int64

In [169]:
N = 112  # more than number of heroes, we have unused ids

In [130]:
X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [159]:
X_pick.shape

(97230, 112)

In [165]:
base_features = test_features.drop(cat_features, axis=1)
base_features = base_features.fillna(0)
X = base_features.values
X_norm = scaler.fit_transform(X)
X_norm.shape

(97230, 92)

In [166]:
X_norm = np.hstack((X_norm, X_pick))
X_norm.shape

(97230, 204)

In [167]:
kf = KFold(n_splits=5, shuffle=True)

res = []
for c in c_range:
    for train_indices, test_indices in kf.split(X, y):
        X_train, X_test = X_norm[train_indices], X_norm[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        clf = LogisticRegression(C=c)
        clf.fit(X_train, y_train)

        pred = clf.predict_proba(X_test)[:, 1]
        score = roc_auc_score(y_test, pred)
        res.append({'roc_auc': score, 'C': c})
    print(c)

1e-05
0.0001
0.001
0.01
0.1
1
10
100


### Result

In [168]:
lin_regr_res_w_cat = pd.DataFrame(res)
lin_regr_res_w_cat.groupby('C').mean().sort_values('roc_auc', ascending=False)

,roc_auc
C,
0.10000,0.752013
100.00000,0.751886
1.00000,0.751860
10.00000,0.751803
0.01000,0.751787
0.00100,0.746181
0.00010,0.725170
0.00001,0.699236


# Test results

## Prepare train data

In [186]:
features = pd.read_csv('features.csv')

In [191]:
train = features.drop(drop_features, axis=1)

data = train[heroes_features]
X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

train_base = train.drop(cat_features, axis=1)
train_base = train_base.fillna(0)
X_train = train_base.values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train.shape

X_train = np.hstack((X_train, X_pick))
X_train.shape

y_train = features['radiant_win'].values

In [196]:
clf = LogisticRegression(C=0.1)  # best on cv
clf.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Prepare test data

In [204]:
features_test = pd.read_csv('features_test.csv')

In [207]:
'radiant_win' in features_test.columns

False

In [210]:
for feature_to_drop in drop_features:
    if feature_to_drop in features_test:
        del features_test[feature_to_drop]
test = features_test

In [211]:
data = test[heroes_features]
X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

test_base = test.drop(cat_features, axis=1)
test_base = test_base.fillna(0)
X_test = test_base.values
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
X_test = np.hstack((X_test, X_pick))
X_test.shape

(17177, 204)

In [212]:
pred = clf.predict_proba(X_test)[:, 1]
#score = roc_auc_score(y_test, pred)

In [215]:
pred

array([ 0.83216894,  0.76340634,  0.19350755, ...,  0.2454362 ,
        0.63587563,  0.44022482])

In [216]:
min(pred), max(pred)

(0.0084676402042699675, 0.99657066046098575)

# Report

## Подход 1: градиентный бустинг "в лоб"

**Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?**

```python
['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']
```
В первые 5 минут матча может не произойти события взятия первой крови: 
```python
['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',]
```
То же с ward_time.

**Как называется столбец, содержащий целевую переменную?**
```python
'radiant_win'
```

**Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? **

train time = **98s** (max)

AUC-ROC = **0.694529**

** Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев? **

Имеет, есть рост качества (см. n_estimators = 40). 
Чтобы ускорить обучения при увеличении количества деревьев можно можно уменьшить глубину деревьев.

## Подход 2: логистическая регрессия

**1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?**

train time = **11.03s** (max)

AUC-ROC = **0.716500** (над всеми исходными признаками) (Выше, чем у град. бустинга)

Логистическая регрессия требует различных предобработок и эвристик чтобы показать приемлемые результаты.
Логистическая регрессия по сравнению с градиентным бустингом работает быстрее.

**2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?**

AUC-ROC = **0.716527** (без категориальных)

Для категориальных признаков в лог. регрессии нужна обработка (bags) т.к. их нельзя сравнивать между собой. Если обработка не используется (1 пункт.) то метод работает чуть лучше без этих параметров.

**3. Сколько различных идентификаторов героев существует в данной игре?**

108 идентификаторов

**4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?**

AUC-ROC = **0.752013** (с правильной обработкой кат. признаков)

Кач-во сильно улучшилось, т.к. начальный выбор героев сильно влияет на исход игры, нужно лишь правильно обработать эти данные.

**5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?**

Лучший алгоритм -- *linear regression, c=0.1, bag of words(cat features)*

min_pred = 0.0084676402042699675

max_pred = 0.99657066046098575